In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
import lxml
import requests


In [3]:
source = requests.get(' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
csv_file = open('toronto_data.csv','w')
csv_writer = csv.writer(csv_file)
table = soup.find('tbody')
df = pd.DataFrame(columns = {'Postcode','Borough','Neighborhood'})
i=0
l = [[]]
n=[]
for tr in table.find_all('tr'):
    csv_writer.writerow([tr.text.split('\n')[1],tr.text.split('\n')[2],tr.text.split('\n')[3]])
csv_file.close()
    
df = pd.read_csv('toronto_data.csv')
df.rename(columns={'Neighbourhood':'Neighborhood', 'Postcode':'PostalCode'}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df_table2 = df.copy()
df_table2 = df.drop([0])
df_table2 = df_table2.drop(df_table2[df_table2['Borough']=='Not assigned'].index)
df_table2 = df_table2.reset_index(drop=True)
df_table2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
df_table2.loc[6]

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Not assigned
Name: 6, dtype: object

In [6]:
df_table2['Neighborhood'] = np.where(df_table2['Neighborhood'] == 'Not assigned', df_table2['Borough'], df_table2['Neighborhood'])

In [7]:
df_table2.loc[6]

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 6, dtype: object

In [8]:
df_table4 = df_table2.copy()

df_table4 =  df_table4.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))

df_table4 = pd.DataFrame(df_table4)
df_table4.head()



,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:


df_table4.shape



(103, 3)

In [11]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
result = pd.merge(df_table4, df_geo, on='PostalCode', how='inner')
result.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
